# AI Community @ Семинар  №5

## Визуализация и t-SNE

### t-Distributed Stochastic Neighbor Embedding (t-SNE)

Источик материалов по t-SNE: [Google TechTalk with Laurens van der Maaten](https://www.youtube.com/watch?v=RJVL80Gg3lA&list=UUtXKDgv1AVoG88PLl8nGXmw)  
Ссылка на статьи, реализации и примеры: https://lvdmaaten.github.io/tsne/

У нас есть $N$ точек из простраства высокой размерности $x_1, x_2, ..., x_N$.  
**Задача:** отобразить эти точки в 2D или 3D пространстве, воспринимаемом человеком, так, чтобы различия между точками в исходном пространстве отражалось в сжатом пространстве.

Так работает PCA на изображениях цифр из датасета MNIST:

![PCA for MNIST](data/pca_mnist.png)
Источник: [High Dimensional Data Visualizing using tSNE](http://yinsenm.github.io/2015/01/01/High-Dimensional-Data-Visualizing-using-tSNE/#toc-Section-2)

Разные классы объектов довольно сильно пересекаются. Если мы уберем цветовую раскраску, то отличить некоторые классы на этой картинке будет просто невозможно.

**Переформулируем немного задачу:** нам необходимо разместить точки из пространства высокой размерности в пространстве низкой размерности так, чтобы близкие объекты в исходном пространстве были близки в новом, и наоборот, далекие объекты в исходном пространстве были так же далеки в новом.

Чтобы измерить похожесть объектов, введем условную вероятность присутствия в определенном месте одной точки при условии присутствия другой:  
$$p_{j|i}=\frac{exp(-\lVert{x_i - x_j}\rVert^2 / 2\sigma_i^2)}{\sum_{k\neq{i}}{exp(-\lVert{x_i - x_k}\rVert^2/2\sigma_i^2)}}$$

![Гауссиана для точки x_i](data/t_sne_gaussian.png)

То же самое сделаем для точек из пространства малой размерности:  
$$q_{j|i}=\frac{exp(-\lVert{y_i - y_j}\rVert^2)}{\sum_{k\neq{i}}{exp(-\lVert{y_i - y_k}\rVert^2)}}$$

На самом деле это выражение взято из оригинальной версии SNE алгоритма. В алгоритме t-SNE вместо нормального распределения используется t-распределение Стьюдента со степению свободы 1 ([Wikipedia](https://en.wikipedia.org/wiki/Student%27s_t-distribution)). Эта модификация сделана для ускорения процесса сходимости, о котором мы скажем позже. Итак,
$$q_{j|i} = \frac{(1 + \lVert{y_i - y_j}\rVert^2)^{-1}}{\sum_{k\neq{i}}{(1 + \lVert{y_i - y_k}\rVert^2)^{-1}}}$$

Мы получили две функции измерения похожести объектов в двух различных пространствах (!). Т.е., если наша модель идеально предсталяет точки в новом пространстве, то $p_{j|i}$ будет равен $q_{j|i}$.

Поскольку точки $y_i$ - это те точки, которые мы собираемся сгенерировать, нам нужно генерировать их по определенному закону. Введем расстояние Кульбака-Лейблера для измерения расстояния между $P_i$ и $Q_i$, где $P_i$ - условное распределение вероятности всех точек $x_j$ кроме $j\neq{i}$, имея $x_i$, а $Q_i$ - условное распределение вероятности всех точек $y_j$ кроме $j\neq{i}$, имея $y_i$:  
$$C = \sum_{i}{KL({P_i}\lVert{Q_i})} = \sum_{i}{\sum_{j}{p_{j|i}log\frac{p_{j|i}}{q_{j|i}}}}$$

$C(y)$ - функция потерь, которую нам необходимо минимизировать. Для нахождения оптимального решения (точек $y_i$) мы можем использовать метод градиентного спуска. После схождения метода, полученные точки можно отрисовать в нашем "человеко-читаемом" пространстве. Результат работы алгоритма t-SNE на датасете MNIST:

![t-SNE for MNIST](data/t_sne_mnist.png)